<img src='https://i.ytimg.com/vi/3H8X8q_XIJU/maxresdefault.jpg' height=300>

In [1]:
%pip install --upgrade --quiet rich tiktoken PyMuPDF langchain langchain-community llama-cpp-python faiss-cpu

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.8/63.8 MB 11.5 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 270.3 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 241.6/241.6 kB 15.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 39.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.5/3.5 MB 59.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.9/15.9 MB 56.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 50.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 64.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.0/27.0 MB 26.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
!wget https://huggingface.co/leliuga/all-MiniLM-L6-v2-GGUF/resolve/main/all-MiniLM-L6-v2.F16.gguf
!wget https://huggingface.co/Qwen/Qwen2-0.5B-Instruct-GGUF/resolve/main/qwen2-0_5b-instruct-q8_0.gguf
!wget https://github.com/fabiomatricardi/How-I-Built-a-Chatbot-that-Crushed-ChatGPT/raw/main/What%20is%20Entrepreneurship%205%20Factors%20Affecting%20Ent.txt
!wget https://huggingface.co/gaianet/Nomic-embed-text-v1.5-Embedding-GGUF/resolve/main/nomic-embed-text-v1.5.f16.gguf

--2024-09-11 23:08:35--  https://huggingface.co/leliuga/all-MiniLM-L6-v2-GGUF/resolve/main/all-MiniLM-L6-v2.F16.gguf
Resolving huggingface.co (huggingface.co)... 13.35.7.57, 13.35.7.38, 13.35.7.5, ...
Connecting to huggingface.co (huggingface.co)|13.35.7.57|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://cdn-lfs-us-1.huggingface.co/repos/68/86/688665eb5b2019706f226f4e2a0cb26c8f210c781549d129dc7f26d630ac2863/797b70c4edf85907fe0a49eb85811256f65fa0f7bf52166b147fd16be2be4662?response-content-disposition=inline%3B+filename*%3DUTF-8%27%27all-MiniLM-L6-v2.F16.gguf%3B+filename%3D%22all-MiniLM-L6-v2.F16.gguf%22%3B&Expires=1726355316&Policy=eyJTdGF0ZW1lbnQiOlt7IkNvbmRpdGlvbiI6eyJEYXRlTGVzc1RoYW4iOnsiQVdTOkVwb2NoVGltZSI6MTcyNjM1NTMxNn19LCJSZXNvdXJjZSI6Imh0dHBzOi8vY2RuLWxmcy11cy0xLmh1Z2dpbmdmYWNlLmNvL3JlcG9zLzY4Lzg2LzY4ODY2NWViNWIyMDE5NzA2ZjIyNmY0ZTJhMGNiMjZjOGYyMTBjNzgxNTQ5ZDEyOWRjN2YyNmQ2MzBhYzI4NjMvNzk3YjcwYzRlZGY4NTkwN2ZlMGE0OWViODU4MTEyNTZmNjVmYTBmN2JmNT

directly from https://python.langchain.com/v0.2/docs/how_to/vectorstore_retriever/

In [3]:
from rich.markdown import Markdown
import warnings
warnings.filterwarnings(action='ignore')
import datetime
from rich.console import Console
console = Console(width=90)
import tiktoken
from time import sleep
encoding = tiktoken.get_encoding("r50k_base") #context_count = len(encoding.encode(yourtext))

from langchain.text_splitter import TokenTextSplitter
from langchain_community.document_loaders import TextLoader
from langchain_community.vectorstores import FAISS
from langchain_community.embeddings import LlamaCppEmbeddings
from llama_cpp import Llama
from langchain_text_splitters import CharacterTextSplitter

In [4]:
embpath = "/content/all-MiniLM-L6-v2.F16.gguf"
embeddings = LlamaCppEmbeddings(model_path=embpath)

llama_model_loader: loaded meta data with 23 key-value pairs and 101 tensors from /content/all-MiniLM-L6-v2.F16.gguf (version GGUF V3 (latest))
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = bert
llama_model_loader: - kv   1:                               general.name str              = all-MiniLM-L6-v2
llama_model_loader: - kv   2:                           bert.block_count u32              = 6
llama_model_loader: - kv   3:                        bert.context_length u32              = 512
llama_model_loader: - kv   4:                      bert.embedding_length u32              = 384
llama_model_loader: - kv   5:                   bert.feed_forward_length u32              = 1536
llama_model_loader: - kv   6:                  bert.attention.head_count u32              = 12
llama_model_loader: - kv   7:          bert.attention.layer_norm_epsilon f

In [ ]:
from llama_cpp import Llama
qwen05b = Llama(
            model_path='/content/qwen2-0_5b-instruct-q8_0.gguf',
            n_gpu_layers=0,
            temperature=0.1,
            top_p = 0.5,
            n_ctx=8192,
            max_tokens=600,
            repeat_penalty=1.7,
            stop=["<|im_end|>","Instruction:","### Instruction:","###<user>","</user>"],
            verbose=False,
            )

In [ ]:
# load a TXT document
loader = TextLoader("/content/What is Entrepreneurship 5 Factors Affecting Ent.txt")
#Create a document and split into chuncks
documents = loader.load()
text_splitter = TokenTextSplitter(chunk_size=250, chunk_overlap=50)
texts = text_splitter.split_documents(documents)
#create the vector store
vectorstore = FAISS.from_documents(texts, embeddings)

In [ ]:
console.print(documents) #this will print the entire document object, without chunks

[
    Document(
        metadata={
            'source': '/content/What is Entrepreneurship 5 Factors Affecting Ent.txt'
        },
        page_content='title: What is Entrepreneurship - 5 Factors Affecting 
Entrepreneurship\xa0\nauthor: Lynsey PT from Adamo Software | Predict | Jun, 2024 | 
Medium\nurl: 
https://medium.com/predict/what-is-entrepreneurship-5-factors-affecting-entrepreneurship-4
aadad3f161e\n\n\nWhat is Entrepreneurship — 5 Factors Affecting Entrepreneurship\nEver 
dreamt of turning your idea into a reality? Explore the world of entrepreneurship! This 
guide dives into what it means to be an entrepreneur.\nHave you ever dreamt of being your 
own boss, building something from the ground up, and making a positive impact on the 
world? If so, then entrepreneurship might be the path for you! But what exactly is 
entrepreneurship?\nEntrepreneurship is the process of identifying opportunities, creating 
new businesses, and taking calculated risks to bring ideas to life. It’s a dynamic and 
challenging journey filled with both immense rewards and potential setbacks.\nI. What is 
entrepreneurship\nThe world of business is brimming with established giants and innovative
upstarts. But what drives the creation of these new ventures? This is where the concept of
entrepreneurship steps in.\nEntrepreneurship is the process of identifying opportunities, 
turning ideas into reality, and creating new businesses. It’s a captivating blend of 
innovation, initiative, and calculated risk-taking. Entrepreneurs are the trailblazers 
who:\nSpot a gap in the market: They have a keen eye for unmet needs or problems waiting 
for solutions.\nCraft unique solutions: They translate those needs into a product, 
service, or business model that addresses the identified gap.\nEmbrace calculated risks: 
They invest their time, resources, and effort into bringing their vision to life, 
understanding the potential rewards and challenges that lie ahead.\nBuild and manage the 
business: They assemble a team, secure funding, navigate the complexities of running a 
successful enterprise, and ensure its growth and sustainability.\nIn essence, 
entrepreneurship is about taking ownership and turning ideas into action. It’s about being
the architect of your own destiny and building something from the ground up.\nII. 
Different types of entrepreneurships\nThe entrepreneurial spirit manifests in various 
forms, each with its unique purpose and approach.\nLet’s delve into some of the most 
common types of entrepreneurship ventures:\nThere are 4 common types of 
entrepreneurship\n1. Startup entrepreneurship\nOften associated with high-growth potential
and disruptive innovation, startup entrepreneurship involves launching a new and 
innovative business. Startups typically focus on technology-driven solutions or entirely 
new business models. Think Uber revolutionizing transportation or Airbnb shaking up the 
hospitality industry. These ventures are often characterized by:\nRapid Growth: Startups 
aim to scale quickly and achieve a dominant market position.\nHigh Risk, High Reward: The 
potential for significant returns is balanced by the inherent risk of failure in a 
competitive landscape.\nFocus on Technology: Many startups leverage technology to create 
innovative products or services.\n2. Small business entrepreneurship\nThe backbone of many
communities, small business entrepreneurship centers around establishing a local business 
that caters to a specific niche or geographic area. Think your friendly neighborhood 
bakery or a local bookstore. These ventures often prioritize:\nStability and 
Sustainability: Small businesses focus on building a loyal customer base and providing 
reliable goods and services within their community.\nCommunity Focus: Understanding and 
catering to the specific needs of the local market is paramount.\nLower Risk Profile: 
While growth is still important, small businesses tend to prioritize stability over rapid 
expansion.\n3. 

In [ ]:
#default is similarity search
retriever = vectorstore.as_retriever(search_type="mmr",search_kwargs={"k": 3})

In [ ]:
docs = retriever.invoke("what to do to get started with entrepreneurship?")

In [ ]:
console.print(docs)

[
    Document(
        metadata={
            'source': '/content/What is Entrepreneurship 5 Factors Affecting Ent.txt'
        },
        page_content=', and operational plan. While it’s a dynamic document that can 
evolve as you learn and grow, a well-crafted business plan will help you stay focused and 
attract potential investors.\nAlso read: Crafting an Effective Product Launch Strategy: 
Best Practices and Examples\n3. Secure Funding\nBootstrapping: Many entrepreneurs start by
bootstrapping their businesses using personal savings, loans from family and friends, or 
crowdfunding platforms.\nSeeking Investors: For ventures requiring significant capital, 
exploring options like angel investors or venture capital firms might be necessary. Be 
prepared to present a compelling pitch that highlights the potential of your business.\n4.
Build Your Team\nAssemble Your A-Team: Surround yourself with talented and passionate 
individuals who complement your skillset. Delegate tasks effectively and foster a 
collaborative environment where everyone feels valued and empowered to contribute.\n5. 
Choose a Business Structure\nSole Proprietorship, Partnership, LLC, or Corporation? The 
legal structure you choose for your business can impact taxes, liabilities, and 
regulations. Consult with a lawyer or accountant to determine the most suitable structure 
for your venture.\n6. Obtain Necessary Licenses and Permits\nComply with Regulations: 
Research the licenses and permits'
    ),
    Document(
        metadata={
            'source': '/content/What is Entrepreneurship 5 Factors Affecting Ent.txt'
        },
        page_content=' for people to develop their skills, pursue their passions, and 
contribute to the community.\n4. Increased Competition: When entrepreneurs enter a market 
with new ideas, they challenge established players to innovate and improve their 
offerings. This healthy competition benefits consumers by leading to lower prices, higher 
quality products and services, and a wider range of choices.\n5. Addressing Social Issues:
Social entrepreneurship tackles some of society’s most pressing challenges. By creating 
businesses focused on social impact, entrepreneurs can address issues like poverty, 
healthcare access, and environmental sustainability. This contributes to a more just and 
equitable society.\n6. Building Resilience: A vibrant entrepreneurial ecosystem with a 
diverse range of businesses makes an economy more resilient to external shocks. New 
ventures can fill gaps left by failing businesses and offer alternative solutions during 
challenging times.\n7. Empowering Individuals: Entrepreneurship allows individuals to take
control of their destinies and build something meaningful. It fosters self-reliance, 
creativity, and problem-solving skills, empowering people to shape their own futures and 
contribute to a more dynamic society.\nIV. Important entrepreneurship skills\nImportant 
entrepreneurship skills\nThe world of entrepreneurship is an exciting adventure, but it’s 
not without its'
    ),
    Document(
        metadata={
            'source': '/content/What is Entrepreneurship 5 Factors Affecting Ent.txt'
        },
        page_content=' navigate the complexities of running a successful enterprise, and 
ensure its growth and sustainability.\nIn essence, entrepreneurship is about taking 
ownership and turning ideas into action. It’s about being the architect of your own 
destiny and building something from the ground up.\nII. Different types of 
entrepreneurships\nThe entrepreneurial spirit manifests in various forms, each with its 
unique purpose and approach.\nLet’s delve into some of the most common types of 
entrepreneurship ventures:\nThere are 4 common types of entrepreneurship\n1. Startup 
entrepreneurship\nOften associated with high-growth potential and disruptive innovation, 
startup entrepreneurship involves launching a new and innovative business. Startups 
typically focus on technology-driven s

#### RESULTS With scores

https://python.langchain.com/v0.2/docs/how_to/add_scores_retriever/

https://python.langchain.com/v0.1/docs/integrations/vectorstores/faiss/

In [ ]:
docs_and_scores = vectorstore.similarity_search_with_score("what is Apple news?",k=3) #or .similarity_search_with_relevance_scores()


llama_print_timings:        load time =     169.52 ms
llama_print_timings:      sample time =       0.00 ms /     1 runs   (    0.00 ms per token,      inf tokens per second)
llama_print_timings: prompt eval time =      15.07 ms /     7 tokens (    2.15 ms per token,   464.56 tokens per second)
llama_print_timings:        eval time =       0.00 ms /     1 runs   (    0.00 ms per token,      inf tokens per second)
llama_print_timings:       total time =      16.47 ms /     8 tokens


In [ ]:
console.print(docs_and_scores)

[
    (
        Document(
            page_content='Apple\'s iOS App Store announces sweeping changes in the 
EU\nhttps://www.axios.com/2024/01/25/apple-app-store-eu-changes\nAuthor: Ashley Gold, 
author of\xa0Axios Pro\n\n\nApple will allow alternative app stores and other new options 
for app developers in order to comply with Europe\'s Digital Markets Act, the company said
Thursday.\n\nDriving the news: Europe\'s major tech competition law, set to go into effect
March 7, requires Apple to loosen its strict rules requiring developers to rely on the App
Store for distribution and payment processing.\n\nApple has long resisted any changes to 
its App Store rules, citing security and privacy. The company also argues its services for
developers are best-in-class and ensure smooth user experiences free of spam or harmful 
content.\n\nWhy it matters: Apple is being forced to make sweeping changes to one of its 
flagship "walled garden" systems in what some smaller companies are seeing as a major 
win.',
            metadata={'source': '/content/Article-edited.txt'}
        ),
        39.76284
    ),
    (
        Document(
            page_content='Go deeper: The iOS changes will include notarization for iOS 
apps, authorization for marketplace developers and disclosures on alternative payments. 
Apple says in the release that if apps opt out of Apple\'s system, the company "has less 
ability to address other risks" like "scam, fraud, abuse" or "objectionable or harmful" 
content.\n\nUsers in the EU will also see a browser choice screen in iOS 17.4 to pick a 
default browser.\n\nThe other side: Epic Games CEO Tim Sweeney said on X: "Apple\'s plan 
to thwart Europe\'s new Digital Markets Act law is a devious new instance of Malicious 
Compliance... Epic has always supported the notion of Apple notarization and malware 
scanning for apps, but we strongly reject Apple\'s twisting this process to undermine 
competition and continue imposing Apple taxes on transactions they\'re not involved in."',
            metadata={'source': '/content/Article-edited.txt'}
        ),
        40.713356
    ),
    (
        Document(
            page_content="What to watch: It's up to the European Commission to determine 
whether Apple's plan fully complies with the DMA. iPhone app developers face a whole new 
world of choices, and time will tell if the changes boost competition and improve 
experiences for users.",
            metadata={'source': '/content/Article-edited.txt'}
        ),
        41.08703
    )
]

### Query the documents and use the LLM to reply

In [ ]:
#Function for QnA over a Context - the context is pure string text
def QwenQnA(question,vectorstore,hits,maxtokens,model):
  """
  basic generation with Qwen2-0.5b-Instruct/ any llama.cpp loaded model
  question -> string
  vectorstore-> a FAISS objects
  hits -> int, the number of results from the similarity search
  maxtokens -> int, number of max tokens to generate
  model -> llama-cpp-python instance // here is Qwen2-0.5b-Instruct AI
  RETURNS output -> str, docs -> list of langchain documents from similarity search
  """
  #docs_and_scores = vectorstore.similarity_search_with_score(question,k=3) #or .similarity_search_with_relevance_scores()
  retriever = vectorstore.as_retriever(search_type="mmr",search_kwargs={"k": hits})
  docs = retriever.invoke(question)
  # pack all the text from the retriever hits and remove carriage return
  context = ''
  for i in docs:
    context += i.page_content
  contesto = context.replace('\n\n','')
  query = question
  import datetime
  start = datetime.datetime.now()
  #create the prompt template
  template = f"""Answer the question based only on the following context:

[context]
{contesto}
[end of context]

Question: {query}

"""
  #structure the prompt into a chat template
  messages = [
              {"role": "system", "content": "You are a Language Model trained to answer questions based solely on the provided text.",},
              {"role": "user", "content": template},
              ]
  with console.status("Qwen-0.5b-Chat AI is working ✅✅✅ ...",spinner="dots12"):
    output = model.create_chat_completion(
                    messages=messages,
                    max_tokens=maxtokens,
                    stop=["</s>","[/INST]","/INST",'<|eot_id|>','<|end|>'],
                    temperature = 0.1,
                    repeat_penalty = 1.4)
  delta = datetime.datetime.now() - start
  console.print(f"[bright_green bold on black]Question: {query}")
  console.print(output["choices"][0]["message"]["content"])
  console.print(f"Completed in: [bold bright_red]{delta}")
  return output["choices"][0]["message"]["content"], docs

In [ ]:
res, notetext = QwenQnA('what is a startup entrepreneurship?', vectorstore,4,500,qwen05b)

Output()

Question: what is a startup entrepreneurship?

A start-up entrepreneur typically focuses on launching new and disruptive innovation in 
the market through technology-driven solutions or entirely new businesses. They aim for 
rapid growth but are also willing to take high risks with potential significant returns, 
balancing this risk against inherent challenges like competition from established players 
within their niche industry. Startups often have a strong focus on creating value by 
leveraging technological advancements while navigating regulatory and operational 
complexities in the competitive environment of startup ecosystems.

Completed in: 0:00:43.189552

In [ ]:
console.print(notetext)

[
    Document(
        metadata={
            'source': '/content/What is Entrepreneurship 5 Factors Affecting Ent.txt'
        },
        page_content=' and challenging journey filled with both immense rewards and 
potential setbacks.\nI. What is entrepreneurship\nThe world of business is brimming with 
established giants and innovative upstarts. But what drives the creation of these new 
ventures? This is where the concept of entrepreneurship steps in.\nEntrepreneurship is the
process of identifying opportunities, turning ideas into reality, and creating new 
businesses. It’s a captivating blend of innovation, initiative, and calculated 
risk-taking. Entrepreneurs are the trailblazers who:\nSpot a gap in the market: They have 
a keen eye for unmet needs or problems waiting for solutions.\nCraft unique solutions: 
They translate those needs into a product, service, or business model that addresses the 
identified gap.\nEmbrace calculated risks: They invest their time, resources, and effort 
into bringing their vision to life, understanding the potential rewards and challenges 
that lie ahead.\nBuild and manage the business: They assemble a team, secure funding, 
navigate the complexities of running a successful enterprise, and ensure its growth and 
sustainability.\nIn essence, entrepreneurship is about taking ownership and turning ideas 
into action. It’s about being the architect of your own destiny and building something 
from the ground up'
    ),
    Document(
        metadata={
            'source': '/content/What is Entrepreneurship 5 Factors Affecting Ent.txt'
        },
        page_content=' failure in a competitive landscape.\nFocus on Technology: Many 
startups leverage technology to create innovative products or services.\n2. Small business
entrepreneurship\nThe backbone of many communities, small business entrepreneurship 
centers around establishing a local business that caters to a specific niche or geographic
area. Think your friendly neighborhood bakery or a local bookstore. These ventures often 
prioritize:\nStability and Sustainability: Small businesses focus on building a loyal 
customer base and providing reliable goods and services within their community.\nCommunity
Focus: Understanding and catering to the specific needs of the local market is 
paramount.\nLower Risk Profile: While growth is still important, small businesses tend to 
prioritize stability over rapid expansion.\n3. Corporate entrepreneurship 
(Intrapreneurship)\nInnovation doesn’t have to happen outside established organizations. 
Intrapreneurship fosters a culture of innovation within existing companies. Intrapreneurs 
are employees who identify and develop new business ideas or products within the larger 
company structure. They might:\nDevelop new product lines or services.\nIdentify new 
market opportunities.\nStreamline internal processes to improve 
efficiency.\nEntrepreneurial ventures benefit from the resources and infrastructure of the
established company, but navigating internal approval processes and overcoming'
    ),
    Document(
        metadata={
            'source': '/content/What is Entrepreneurship 5 Factors Affecting Ent.txt'
        },
        page_content=', and operational plan. While it’s a dynamic document that can 
evolve as you learn and grow, a well-crafted business plan will help you stay focused and 
attract potential investors.\nAlso read: Crafting an Effective Product Launch Strategy: 
Best Practices and Examples\n3. Secure Funding\nBootstrapping: Many entrepreneurs start by
bootstrapping their businesses using personal savings, loans from family and friends, or 
crowdfunding platforms.\nSeeking Investors: For ventures requiring significant capital, 
exploring options like angel investors or venture capital firms might be necessary. Be 
prepared to present a compelling pitch that highlights the potential of your business.\n4.
Build Your Team\nAssemble Your A-Team: Surround yourself with talented and passi

In [ ]:
res, notetext = QwenQnA('what is the Small business entrepreneurship?', vectorstore,4,500,qwen05b)

Output()

Question: what is the Small business entrepreneurship?

Small Business Entrepreneurship centers on establishing local small-scale enterprises that
cater to specific geographic areas, focusing primarily upon stability (stability) & 
sustainability (sustainability), community focus (community), lower risk profile (risk). 
It also involves understanding and catering towards a particular market niche. The venture
often prioritizes the establishment of an established business structure within its 
locality for long-term growth potential in terms of resources like infrastructure or 
capital needed to expand operations, but navigating internal approval processes can be 
challenging due to regulatory requirements related to title issues such as "What is 
Entrepreneurship - 5 Factors Affecting Entrepreneurship"

Completed in: 0:00:46.440196

In [ ]:
console.print(notetext)

[
    Document(
        page_content=' failure in a competitive landscape.\nFocus on Technology: Many 
startups leverage technology to create innovative products or services.\n2. Small business
entrepreneurship\nThe backbone of many communities, small business entrepreneurship 
centers around establishing a local business that caters to a specific niche or geographic
area. Think your friendly neighborhood bakery or a local bookstore. These ventures often 
prioritize:\nStability and Sustainability: Small businesses focus on building a loyal 
customer base and providing reliable goods and services within their community.\nCommunity
Focus: Understanding and catering to the specific needs of the local market is 
paramount.\nLower Risk Profile: While growth is still important, small businesses tend to 
prioritize stability over rapid expansion.\n3. Corporate entrepreneurship 
(Intrapreneurship)\nInnovation doesn’t have to happen outside established organizations. 
Intrapreneurship fosters a culture of innovation within existing companies. Intrapreneurs 
are employees who identify and develop new business ideas or products within the larger 
company structure. They might:\nDevelop new product lines or services.\nIdentify new 
market opportunities.\nStreamline internal processes to improve 
efficiency.\nEntrepreneurial ventures benefit from the resources and infrastructure of the
established company, but navigating internal approval processes and overcoming',
        metadata={
            'source': '/content/What is Entrepreneurship 5 Factors Affecting Ent.txt'
        }
    ),
    Document(
        page_content='title: What is Entrepreneurship - 5 Factors Affecting 
Entrepreneurship\xa0\nauthor: Lynsey PT from Adamo Software | Predict | Jun, 2024 | 
Medium\nurl: 
https://medium.com/predict/what-is-entrepreneurship-5-factors-affecting-entrepreneurship-4
aadad3f161e\n\n\nWhat is Entrepreneurship — 5 Factors Affecting Entrepreneurship\nEver 
dreamt of turning your idea into a reality? Explore the world of entrepreneurship! This 
guide dives into what it means to be an entrepreneur.\nHave you ever dreamt of being your 
own boss, building something from the ground up, and making a positive impact on the 
world? If so, then entrepreneurship might be the path for you! But what exactly is 
entrepreneurship?\nEntrepreneurship is the process of identifying opportunities, creating 
new businesses, and taking calculated risks to bring ideas to life. It’s a dynamic and 
challenging journey filled with both immense rewards and potential setbacks.\nI. What is 
entrepreneurship\nThe world of business is brimming with established giants and innovative
upstarts. But what drives the creation of these new ventures? This is where the concept',
        metadata={
            'source': '/content/What is Entrepreneurship 5 Factors Affecting Ent.txt'
        }
    ),
    Document(
        page_content=', and operational plan. While it’s a dynamic document that can 
evolve as you learn and grow, a well-crafted business plan will help you stay focused and 
attract potential investors.\nAlso read: Crafting an Effective Product Launch Strategy: 
Best Practices and Examples\n3. Secure Funding\nBootstrapping: Many entrepreneurs start by
bootstrapping their businesses using personal savings, loans from family and friends, or 
crowdfunding platforms.\nSeeking Investors: For ventures requiring significant capital, 
exploring options like angel investors or venture capital firms might be necessary. Be 
prepared to present a compelling pitch that highlights the potential of your business.\n4.
Build Your Team\nAssemble Your A-Team: Surround yourself with talented and passionate 
individuals who complement your skillset. Delegate tasks effectively and foster a 
collaborative environment where everyone feels valued and empowered to contribute.\n5. 
Choose a Business Structure\nSole Proprietorship, Partnership, LLC, or Corporation? The 
legal structure you choose for you

In [ ]:
res, notetext = QwenQnA('what to do to get started with entrepreneurship?', vectorstore,5,500,qwen05b)

Output()

Question: what to do to get started with entrepreneurship?

To start a business venture or entrepreneurial endeavor:

1) Identify an opportunity that aligns well with one's skills, interests, passions.

2) Develop strategies for creating value through innovation. This might involve developing
unique products/services and/or building new businesses from scratch based on existing 
ones (startups).

3) Build the necessary resources to launch your business venture or entrepreneurial 
endeavor: this could include funding sources like loans/loans with interest-free terms; 
securing a startup loan, which can be used for initial capital investment.

4) Develop marketing strategies that attract and retain customers. This might involve 
creating an online presence (e.g., social media), developing brand awareness through 
advertising campaigns, or leveraging partnerships to reach new audiences effectively.

5) Establish clear goals: define what you want your business venture/entrepreneurship to 
achieve in terms of revenue streams, market share growth objectives etc.

6) Secure funding sources. This might include seeking loans from banks and investors 
(startups), crowdfunding platforms like Kickstarter for small businesses or grants 
provided by government agencies that support entrepreneurship programs such as the Small 
Business Administration's SBA loan program.

7) Build a strong team: recruit talented individuals with complementary skills to 
complement your own skillset, establish clear roles within teams based on their expertise.

8) Develop and manage business operations effectively:

- Create an operational plan for managing day-to-day tasks.
  
9) Secure funding through various sources (startups/loans; crowdfunding platforms).

10) Monitor progress regularly with a focus on metrics like revenue growth, customer 
acquisition costs etc.

Remember that entrepreneurship is not just about starting your own venture but also being 
able to adapt and grow as the market evolves.

Completed in: 0:01:28.099205

In [ ]:
console.print(notetext)